In [1]:
from collections import OrderedDict
from conllu import parse
from enum import Enum

PATH = "/Users/serg/projects/UD_Ukrainian-IU"

with open(PATH + "/uk_iu-ud-train.conllu", "r") as f:
    data = f.read()

trees = parse(data)

In [6]:
tree = trees[0]
# debug mode
#for node in tree:
#    head = node["head"]
#    print("{} <-- {}".format(node["form"],
#                             tree[head - 1]["form"]
#                             if head > 0 else "root"))
    
def deps(tree):
    return ["{} <-- {}".format(node["form"], tree[int(node['head']) - 1]["form"]\
                               if int(node['head']) > 0 else "root") for node in tree]

print_deps(tree)

['У <-- домі',
 'домі <-- була',
 'римського <-- патриція',
 'патриція <-- домі',
 'Руфіна <-- патриція',
 'була <-- root',
 'прегарна <-- фреска',
 'фреска <-- була',
 ', <-- зображення',
 'зображення <-- фреска',
 'Венери <-- зображення',
 'та <-- Адоніса',
 'Адоніса <-- Венери',
 '. <-- була']

In [3]:
SHIFT = 1
REDUCE = 2
LEFT_ARC = 3
RIGHT_ARC = 4
    

In [27]:
ROOT = OrderedDict([('id', 0), ('form', 'ROOT'), ('lemma', 'ROOT'), ('upostag', 'ROOT'),
                    ('xpostag', None), ('feats', None), ('head', None), ('deprel', None),
                    ('deps', None), ('misc', None)])

class State:
    def __init__(self, sent):
        self.stack = []
        self.stack.append(ROOT)
        self.queue = []
        self.queue.extend(sent)
        self.relations = []
        
    def non_final(self):
        return len(self.stack) > 0 or len(self.queue) > 0
    
    def apply(self, action):
        if action == SHIFT:
            self.stack.append(self.queue.pop(0))
        elif action == REDUCE:
            self.stack.pop()
        elif action == LEFT_ARC:
            self.relations.append((self.stack[-1]["id"], self.queue[0]["id"]))
            self.stack.pop()
        elif action == RIGHT_ARC:
            self.relations.append((self.queue[0]["id"], self.stack[-1]["id"]))
            self.stack.append(self.queue.pop(0))
        else:
            assert False
    
    def oracle(self):
        """
        Make a decision on the right action to do.
        """
        top_stack = self.stack[-1] if len(self.stack) > 0 else None
        top_queue = self.queue[0] if len(self.queue) > 0 else None
        # check if both stack and queue are non-empty
        if top_stack and not top_queue:
            return REDUCE
        # check if there are any clear dependencies
        elif top_queue["head"] == top_stack["id"]:
            return RIGHT_ARC
        elif top_stack["head"] == top_queue["id"]:
            return LEFT_ARC
        # check if we can reduce the top of the stack
        elif top_stack["id"] in [i[0] for i in self.relations] and \
             (top_queue["head"] < top_stack["id"] or \
              [s for s in self.stack if s["head"] == top_queue["id"]]):
            return REDUCE
        # default option
        else:
            return SHIFT
    

def dependency_parse(sent):
    state = State(sent)
    while state.non_final():
        op = state.oracle()
        state.apply(op)
    return state

    

In [30]:
print(deps(tree))
print(list(map(lambda x: str(x[0]) + ' <-- ' + str(x[1]), dependency_parse(tree[:]).relations)))

['У <-- домі', 'домі <-- була', 'римського <-- патриція', 'патриція <-- домі', 'Руфіна <-- патриція', 'була <-- root', 'прегарна <-- фреска', 'фреска <-- була', ', <-- зображення', 'зображення <-- фреска', 'Венери <-- зображення', 'та <-- Адоніса', 'Адоніса <-- Венери', '. <-- була']
['1 <-- 2', '3 <-- 4', '4 <-- 2', '5 <-- 4', '2 <-- 6', '6 <-- 0', '7 <-- 8', '8 <-- 6', '9 <-- 10', '10 <-- 8', '11 <-- 10', '12 <-- 13', '13 <-- 11', '14 <-- 6']


In [33]:
print(len(deps(tree)))
print(len(list(map(lambda x: str(x[0]) + ' <-- ' + str(x[1]), dependency_parse(tree[:]).relations))))

14
14


In [ ]:
def extract_sent_features(sent):
    features = []
    state = State(sent)
    while state.non_final
        fdic = {}
        op = state.oracle()
        fdic['form']
        state.apply(op)